<a href="https://colab.research.google.com/github/noheat61/my.Avatar-AI/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# **SetUp**

In [ ]:
# git clone
import os

!git clone https://github.com/noheat61/my.Avatar-AI avatar-ai
os.chdir(f'./avatar-ai')

In [ ]:
!pip install face-alignment ninja kornia yacs

In [ ]:
!pip install fvcore chumpy

import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

In [ ]:
# 필요한 대용량 파일 다운로드
!python download_cartoon_data.py
!bash download_deca_model.sh

In [ ]:
import os

if not os.path.exists("cartoon_image"):
    os.mkdir("cartoon_image")
if not os.path.exists("avatar"):
    os.mkdir("avatar")

# **Run with Your Own Image**

In [ ]:
from google.colab import files
uploaded = files.upload()
image_name = list(uploaded.keys())[0]
print(image_name)

In [ ]:
from infer import Model2D, Model3D

model2D = Model2D()
model3D = Model3D()

cartoon_path = "cartoon_image/"
avatar_path = "avatar/"

In [ ]:
model2D.inference(input_path=image_name, output_path=cartoon_path, make_all=False, style="DISNEY") # style: DISNEY, 여신강림

In [ ]:
model3D.inference(input_path=cartoon_path, output_path=avatar_path)

# **Download results**

In [ ]:
from google.colab import files

!zip -r avatar_results.zip avatar
files.download("avatar_results.zip")